# Data Preprocessing for the Clutch<sup>2</sup> Model

For the Model, we need the Shots Dataset for 2007-2024. We will also need players and teams for these seasons. Once these datasets are created, we can combine them to create the necessary dataframes for the WPA model.

Recall the required data for the Pettigrew WP model are:
- Strength Data(5on5, 4on5, 5on4, etc)
- Scoring Rates for a Team (Pull Aggregated Team Data) at each Strength
- Time Remaining in the Game
- The Score Differential (Goals For Per Game - GFg, Goals Allowed Per Game - GAg)
- Time Remaining for a Penalty

And the data for the Ryder Estimation are:
- Goal Differential
- Time Remaining
- GFg and GAg



In [1]:
import os
import pandas as pd
import polars as pl
from tqdm import tqdm
from pathlib import Path
import requests
import pandas as pd
from collections import defaultdict
from typing import Tuple, Union
import numpy as np
from math import sqrt, exp
from scipy.stats import poisson

# 1. Create the Team, Player, Line and Shot datasets

This data comes from Peter Tanner's [MoneyPuck](<>) site, and was renamed to the format `{category}_{season}.csv`. For example, the team data from 2024 would be `teams_2024.csv`.

In [2]:
# Paths to data directories
teams_path = '/Users/dB/Documents/repos/github/hockey-clutch2-model/data/teams_data/'
skaters_path = '/Users/dB/Documents/repos/github/hockey-clutch2-model/data/skaters_data/'
lines_path = '/Users/dB/Documents/repos/github/hockey-clutch2-model/data/lines_data/'

# Starting and Ending Dates
start = 2008
end = 2024

### 1.a. Creating the Teams Data

**Important Notes for People Recreating This:** 

- The 2018 Teams dataset has the columns `penalitiesFor` and `penalitiesAgainst` instead of `penaltiesFor` and `penaltiesAgainst`, this needs to be fixed for the program to run!

In [3]:
def process_teams_data(teams_path):
    dfs = [] # store dfs here
    
    # First pass: collect schema information
    all_columns = set()
    for team_file in os.listdir(teams_path):
        if not team_file.endswith('.csv'):
            continue
        df = pl.scan_csv(f"{teams_path}/{team_file}").collect()
        all_columns.update(df.columns)
    
    print(f"Total unique columns found: {len(all_columns)}")
    
    # Second pass: load and standardize each file
    for team_file in tqdm(os.listdir(teams_path)):
        if not team_file.endswith('.csv'):
            continue
            
        try:
            # Extract season from filename
            season = int(team_file.split('_')[1].split('.')[0])
            
            # Read CSV with more flexible schema handling
            df = pl.scan_csv(f"{teams_path}/{team_file}").collect()
            
            # Add missing columns with null values
            missing_cols = all_columns - set(df.columns)
            for col in missing_cols:
                df = df.with_columns(pl.lit(None).alias(col))
            
            # Add season column - Redundant but whatever
            df = df.with_columns(pl.lit(season).alias('season'))
            
            # Cast all numeric cols to floats
            for col in df.columns:
                if df[col].dtype in [pl.Int64, pl.Float64]:
                    df = df.with_columns(pl.col(col).cast(pl.Float64))
            
            dfs.append(df)
            
        except Exception as e:
            print(f"Error processing {team_file}: {str(e)}")
    
    if not dfs:
        print("No valid team files were processed")
        return None
    
    # Combine all DataFrames
    teams = pl.concat(dfs, how="vertical")
    
    # Sort by season and team
    teams = teams.sort(["season", "team"])
    
    # Create output directory if it doesn't exist
    os.makedirs("processed_data", exist_ok=True)
    
    # Save to parquet
    teams.write_parquet(f'{teams_path[:-11]}processed_data/teams.parquet')
    print("parquet file written to data/processed_data/teams.parquet")
    
    return teams

In [4]:
teams = process_teams_data(teams_path)

if teams is not None:
    print("\nFirst few rows of processed data:")
    print(teams.head())
    print("\nDataFrame schema:")
    print(teams.schema)
    print("\nShape of final DataFrame:")
    print(teams.shape)

Total unique columns found: 107


100%|██████████| 17/17 [00:01<00:00, 12.72it/s]

parquet file written to data/processed_data/teams.parquet

First few rows of processed data:
shape: (5, 107)
┌──────┬────────┬──────┬──────────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ team ┆ season ┆ name ┆ team_duplica ┆ … ┆ reboundxGoal ┆ totalShotCr ┆ scoreAdjust ┆ scoreFlurry │
│ ---  ┆ ---    ┆ ---  ┆ ted_0        ┆   ┆ sAgainst     ┆ editAgainst ┆ edTotalShot ┆ AdjustedTot │
│ str  ┆ i32    ┆ str  ┆ ---          ┆   ┆ ---          ┆ ---         ┆ CreditAg…   ┆ alShotCr…   │
│      ┆        ┆      ┆ str          ┆   ┆ f64          ┆ f64         ┆ ---         ┆ ---         │
│      ┆        ┆      ┆              ┆   ┆              ┆             ┆ f64         ┆ f64         │
╞══════╪════════╪══════╪══════════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ ANA  ┆ 2008   ┆ ANA  ┆ ANA          ┆ … ┆ 35.0         ┆ 219.8       ┆ 219.52      ┆ 212.32      │
│ ANA  ┆ 2008   ┆ ANA  ┆ ANA          ┆ … ┆ 20.34        ┆ 124.43      ┆ 124.14    

In [5]:
teams.null_count()

team,season,name,team_duplicated_0,position,situation,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penaltyMinutesFor,faceOffsWonFor,…,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 1.b. Creating the Skaters Data

In [6]:
def process_skaters_data(skaters_path):
    dfs = []  # store dfs here
    
    # First pass: collect schema information
    all_columns = set()
    for skater_file in os.listdir(skaters_path):
        if not skater_file.endswith('.csv'):
            continue
        df = pl.scan_csv(f"{skaters_path}/{skater_file}").collect()
        all_columns.update(df.columns)
    
    print(f"Total unique columns found: {len(all_columns)}")
    
    # Second pass: load and standardize each file
    for skater_file in tqdm(os.listdir(skaters_path)):
        if not skater_file.endswith('.csv'):
            continue
            
        try:
            # Extract season from filename
            season = int(skater_file.split('_')[1].split('.')[0])
            
            # Read CSV with more flexible schema handling
            df = pl.scan_csv(f"{skaters_path}/{skater_file}").collect()
            
            # Add missing columns with null values
            missing_cols = all_columns - set(df.columns)
            for col in missing_cols:
                df = df.with_columns(pl.lit(None).alias(col))
            
            # Add season column - Redundant but whatever
            df = df.with_columns(pl.lit(season).alias('season'))
            
            # Cast all numeric cols to floats
            for col in df.columns:
                if df[col].dtype in [pl.Int64, pl.Float64]:
                    df = df.with_columns(pl.col(col).cast(pl.Float64))
                    
            dfs.append(df)
            
        except Exception as e:
            print(f"Error processing {skater_file}: {str(e)}")
    
    if not dfs:
        print("No valid skater files were processed")
        return None
        
    # Combine all DataFrames
    skaters = pl.concat(dfs, how="vertical")
    
    # Sort by season and player
    skaters = skaters.sort(["season", "playerId"])
    
    # Create output directory if it doesn't exist
    os.makedirs("processed_data", exist_ok=True)
    
    # Save to parquet
    skaters.write_parquet(f'{skaters_path[:-13]}processed_data/skaters.parquet')
    print("parquet file written to data/processed_data/skaters.parquet")
    
    return skaters


In [7]:
skaters = process_skaters_data(skaters_path)

if skaters is not None:
    print("\nFirst few rows of processed data:")
    print(teams.head())
    print("\nDataFrame schema:")
    print(teams.schema)
    print("\nShape of final DataFrame:")
    print(teams.shape)

Total unique columns found: 154


100%|██████████| 17/17 [00:28<00:00,  1.70s/it]


parquet file written to data/processed_data/skaters.parquet

First few rows of processed data:
shape: (5, 107)
┌──────┬────────┬──────┬──────────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ team ┆ season ┆ name ┆ team_duplica ┆ … ┆ reboundxGoal ┆ totalShotCr ┆ scoreAdjust ┆ scoreFlurry │
│ ---  ┆ ---    ┆ ---  ┆ ted_0        ┆   ┆ sAgainst     ┆ editAgainst ┆ edTotalShot ┆ AdjustedTot │
│ str  ┆ i32    ┆ str  ┆ ---          ┆   ┆ ---          ┆ ---         ┆ CreditAg…   ┆ alShotCr…   │
│      ┆        ┆      ┆ str          ┆   ┆ f64          ┆ f64         ┆ ---         ┆ ---         │
│      ┆        ┆      ┆              ┆   ┆              ┆             ┆ f64         ┆ f64         │
╞══════╪════════╪══════╪══════════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ ANA  ┆ 2008   ┆ ANA  ┆ ANA          ┆ … ┆ 35.0         ┆ 219.8       ┆ 219.52      ┆ 212.32      │
│ ANA  ┆ 2008   ┆ ANA  ┆ ANA          ┆ … ┆ 20.34        ┆ 124.43      ┆ 124.14  

### 1.c. Creating the Lines Data

In [8]:
def process_lines_data(lines_path):
    dfs = []  # store dfs here
    
    # First pass: collect schema information
    all_columns = set()
    for line_file in os.listdir(lines_path):
        if not line_file.endswith('.csv'):
            continue
        df = pl.scan_csv(f"{lines_path}/{line_file}").collect()
        all_columns.update(df.columns)
    
    print(f"Total unique columns found: {len(all_columns)}")
    
    # Second pass: load and standardize each file
    for line_file in tqdm(os.listdir(lines_path)):
        if not line_file.endswith('.csv'):
            continue
            
        try:
            # Extract season from filename
            season = int(line_file.split('_')[1].split('.')[0])
            
            # Read CSV with more flexible schema handling
            df = pl.scan_csv(f"{lines_path}/{line_file}").collect()
            
            # Add missing columns with null values
            missing_cols = all_columns - set(df.columns)
            for col in missing_cols:
                df = df.with_columns(pl.lit(None).alias(col))
            
            # Add season column - Redundant but whatever
            df = df.with_columns(pl.lit(season).alias('season'))
            
            # Cast all numeric cols to floats
            for col in df.columns:
                if df[col].dtype in [pl.Int64, pl.Float64]:
                    df = df.with_columns(pl.col(col).cast(pl.Float64))
                    
            dfs.append(df)
            
        except Exception as e:
            print(f"Error processing {line_file}: {str(e)}")
    
    if not dfs:
        print("No valid line files were processed")
        return None
        
    # Combine all DataFrames
    lines = pl.concat(dfs, how="vertical")
    
    # Sort by season and line
    lines = lines.sort(["season", "lineId"])
    
    # Create output directory if it doesn't exist
    os.makedirs("processed_data", exist_ok=True)
    
    # Save to parquet
    lines.write_parquet(f'{lines_path[:-11]}processed_data/lines.parquet')
    print("parquet file written to data/processed_data/lines.parquet")
    
    return lines

In [9]:
lines = process_lines_data(lines_path)

if lines is not None:
    print("\nFirst few rows of processed data:")
    print(teams.head())
    print("\nDataFrame schema:")
    print(teams.schema)
    print("\nShape of final DataFrame:")
    print(teams.shape)

Total unique columns found: 108


100%|██████████| 17/17 [00:13<00:00,  1.28it/s]


parquet file written to data/processed_data/lines.parquet

First few rows of processed data:
shape: (5, 107)
┌──────┬────────┬──────┬──────────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ team ┆ season ┆ name ┆ team_duplica ┆ … ┆ reboundxGoal ┆ totalShotCr ┆ scoreAdjust ┆ scoreFlurry │
│ ---  ┆ ---    ┆ ---  ┆ ted_0        ┆   ┆ sAgainst     ┆ editAgainst ┆ edTotalShot ┆ AdjustedTot │
│ str  ┆ i32    ┆ str  ┆ ---          ┆   ┆ ---          ┆ ---         ┆ CreditAg…   ┆ alShotCr…   │
│      ┆        ┆      ┆ str          ┆   ┆ f64          ┆ f64         ┆ ---         ┆ ---         │
│      ┆        ┆      ┆              ┆   ┆              ┆             ┆ f64         ┆ f64         │
╞══════╪════════╪══════╪══════════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ ANA  ┆ 2008   ┆ ANA  ┆ ANA          ┆ … ┆ 35.0         ┆ 219.8       ┆ 219.52      ┆ 212.32      │
│ ANA  ┆ 2008   ┆ ANA  ┆ ANA          ┆ … ┆ 20.34        ┆ 124.43      ┆ 124.14    

## 2. Shot Data

To classify Clutch<sup>2</sup> goals, we need to read in data of all shots taken (again, from [Moneypuck](<>)). We will read in the shot data and add dates of the game each shot took place in. This will let us connect the shot's clutch-ness to the team's stats (like Corsi, Fenwick, etc.) at the time, providing a more accurate model than the season-level aggregation of those same statistics.

### 2.a. Combining 2007-2023 and 2024 Shot Data

The complete shot dataset is only updated at season's end, so to account for this season, we need to concatenate the 2024 season's shots to the 2007-2023 dataset.

In [10]:
# Set filepaths+names
historical_shots_file = '/Users/dB/Documents/repos/github/hockey-clutch2-model/data/shot_data/shots_2007-2023.csv'
current_shots_file = '/Users/dB/Documents/repos/github/hockey-clutch2-model/data/shot_data/shots_2024.csv'

# Get Historical and Current Shots
historical_shots = pl.scan_csv(historical_shots_file).collect()
current_shots = pl.scan_csv(current_shots_file).collect()

# Cast all Numeric Variables to Floats
for col in historical_shots.columns:
                if historical_shots[col].dtype in [pl.Int64, pl.Float64]:
                    historical_shots = historical_shots.with_columns(pl.col(col).cast(pl.Float64))

for col in current_shots.columns:
                if current_shots[col].dtype in [pl.Int64, pl.Float64]:
                    current_shots = current_shots.with_columns(pl.col(col).cast(pl.Float64))

I am borrowing this function from `shots-concat-cleaner.ipynb` in this repository. This will merge the current and historical shots:


Compare two dataframes to check if they have the same columns, regardless of order.
Returns a tuple containing:
- Boolean indicating if columns match exactly
- Set of columns in df1 but not in df2 (if any)
- Set of columns in df2 but not in df1 (if any)
    
Parameters:
df1 : pandas.DataFrame
    First DataFrame to compare
df2 : pandas.DataFrame
    Second DataFrame to compare
        
Returns:

tuple : (bool, set, set)
- True if columns match exactly, False otherwise
- Columns unique to df1
- Columns unique to df2
    
Runtime Complexity: O(n) where n is the total number of columns
Space Complexity: O(n) to store the column sets

In [11]:
def compare_dataframe_columns(df1, df2):
    # Convert column lists to sets for O(1) lookup time
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    
    # Find columns unique to each DataFrame
    cols_only_in_df1 = cols1 - cols2
    cols_only_in_df2 = cols2 - cols1
    
    # Check if the columns match exactly
    columns_match = len(cols_only_in_df1) == 0 and len(cols_only_in_df2) == 0
    
    return columns_match, cols_only_in_df1, cols_only_in_df2

In [12]:
compare_dataframe_columns(historical_shots, current_shots)

(False,
 set(),
 {'homeTeamScore',
  'homeWinProbability',
  'shotGoalProbability',
  'shotPlayContinued',
  'timeBetweenEvents',
  'timeLeft'})

In [13]:
def append_matching_columns(df1: pl.DataFrame, df2: pl.DataFrame) -> pl.DataFrame:
    # Get columns that are common to both DataFrames
    common_cols = [col for col in df1.columns if col in df2.columns]

    # Select only matching columns from df2
    df2_selected = df2.select(common_cols)

    # Add missing columns with null values to df2 to align with df1
    missing_cols = [col for col in df1.columns if col not in df2.columns]
    df2_aligned = df2_selected.with_columns([pl.lit(None).alias(col) for col in missing_cols])

    # Reorder df2 to match df1's column order
    df2_reordered = df2_aligned.select(df1.columns)

    # Concatenate the two DataFrames
    combined_df = pl.concat([df1, df2_reordered], how="vertical")

    return combined_df


In [14]:
shots = append_matching_columns(historical_shots, current_shots)

In [15]:
shots.head()

shotID,homeTeamCode,awayTeamCode,season,isPlayoffGame,game_id,homeTeamWon,id,time,timeUntilNextEvent,timeSinceLastEvent,period,team,location,event,goal,shotPlayContinuedOutsideZone,shotPlayContinuedInZone,shotGoalieFroze,shotPlayStopped,shotGeneratedRebound,homeTeamGoals,awayTeamGoals,xCord,yCord,xCordAdjusted,yCordAdjusted,shotAngle,shotAngleAdjusted,shotAnglePlusRebound,shotAngleReboundRoyalRoad,shotDistance,shotType,shotOnEmptyNet,shotRebound,shotAnglePlusReboundSpeed,shotRush,…,defendingTeamDefencemenOnIce,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,offWing,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedYCord,arenaAdjustedYCordAbs,timeDifferenceSinceChange,averageRestDifference,xGoal,xFroze,xRebound,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xShotWasOnGoal,isHomeTeam,shotWasOnGoal,teamCode,arenaAdjustedXCordABS
f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
0.0,"""TBL""","""NSH""",2023.0,0.0,20001.0,1.0,10.0,61.0,9.0,3.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,59.0,-26.0,59.0,-26.0,-40.914383,40.914383,0.0,0.0,39.698866,"""WRIST""",0.0,0.0,0.0,0.0,…,2.0,30.0,28.666667,32.0,33.0,31.0,33.0,26.0,26.0,31.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,0.0,39.698866,59.0,-26.0,26.0,-6.0,-3.4,0.035813,0.249174,0.035816,0.37931,0.277143,0.0227439,0.75132,1.0,1.0,"""TBL""",59.0
1.0,"""TBL""","""NSH""",2023.0,0.0,20001.0,1.0,11.0,70.0,26.0,9.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,8.0,81.0,8.0,45.0,45.0,85.914383,1.0,11.313708,"""TIP""",0.0,0.0,9.546043,0.0,…,2.0,39.0,37.666667,41.0,42.0,40.0,42.0,35.0,35.0,40.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,1.0,11.313708,81.0,8.0,8.0,-6.0,-3.4,0.057705,0.107945,0.051285,0.464358,0.298119,0.020588,0.589712,1.0,1.0,"""TBL""",81.0
2.0,"""TBL""","""NSH""",2023.0,0.0,20001.0,1.0,13.0,107.0,2.0,11.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,55.0,30.0,55.0,30.0,41.423666,41.423666,0.0,0.0,45.343136,"""SNAP""",0.0,0.0,0.0,0.0,…,2.0,23.8,22.333333,26.0,26.0,25.0,26.0,19.0,19.0,26.0,23.8,22.333333,26.0,26.0,25.0,26.0,19.0,19.0,26.0,0.0,45.343136,55.0,30.0,30.0,0.0,-3.8,0.016126,0.236193,0.030269,0.390417,0.301375,0.02562,0.785765,1.0,1.0,"""TBL""",55.0
3.0,"""TBL""","""NSH""",2023.0,0.0,20001.0,1.0,19.0,177.0,35.0,32.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0,-30.0,58.0,-30.0,-44.060809,44.060809,0.0,0.0,43.139309,"""WRIST""",0.0,0.0,0.0,0.0,…,2.0,40.0,36.666667,45.0,47.0,47.0,46.0,26.0,26.0,44.0,40.0,36.666667,45.0,47.0,47.0,46.0,26.0,26.0,44.0,1.0,43.139309,58.0,-30.0,30.0,2.0,-3.4,0.0167967,0.198635,0.0336855,0.438052,0.291724,0.021107,0.724471,1.0,1.0,"""TBL""",58.0
4.0,"""TBL""","""NSH""",2023.0,0.0,20001.0,1.0,21.0,231.0,29.0,19.0,1.0,"""AWAY""","""HOMEZONE""","""MISS""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.0,34.0,64.0,-34.0,-53.673174,53.673174,0.0,0.0,42.201896,"""WRIST""",0.0,0.0,0.0,0.0,…,2.0,44.4,47.0,40.5,48.0,48.0,44.0,37.0,46.0,37.0,44.4,47.0,40.5,48.0,48.0,44.0,37.0,46.0,37.0,0.0,42.201896,-64

Now that we have a concatenated shots DataFrame, we need to assign a custom game ID to each game. Moneypuck's datasets reset each season so the first `game_id` in each dataset is 20001. We first need to cast `season` and `game_id` back to ints, then merge the numbers into one big ID, `composite_game_id`.

These had to be broken into 2 cells for some reason idk

In [16]:
shots = shots.with_columns(
    pl.when(pl.col("game_id").cast(pl.Int64) < 100000)
    .then((pl.col("season").cast(pl.Int64) * 10**6 + pl.col("game_id").cast(pl.Int64) + 10**5))
    .otherwise((pl.col("season").cast(pl.Int64) * 10**6 + pl.col("game_id").cast(pl.Int64)))
    .alias("composite_game_id")
)

shots = shots.with_columns(
    [
        pl.col("season").cast(pl.Int64),
        pl.col("game_id").cast(pl.Int64)
    ]
)

In [17]:
shots

shotID,homeTeamCode,awayTeamCode,season,isPlayoffGame,game_id,homeTeamWon,id,time,timeUntilNextEvent,timeSinceLastEvent,period,team,location,event,goal,shotPlayContinuedOutsideZone,shotPlayContinuedInZone,shotGoalieFroze,shotPlayStopped,shotGeneratedRebound,homeTeamGoals,awayTeamGoals,xCord,yCord,xCordAdjusted,yCordAdjusted,shotAngle,shotAngleAdjusted,shotAnglePlusRebound,shotAngleReboundRoyalRoad,shotDistance,shotType,shotOnEmptyNet,shotRebound,shotAnglePlusReboundSpeed,shotRush,…,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,offWing,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedYCord,arenaAdjustedYCordAbs,timeDifferenceSinceChange,averageRestDifference,xGoal,xFroze,xRebound,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xShotWasOnGoal,isHomeTeam,shotWasOnGoal,teamCode,arenaAdjustedXCordABS,composite_game_id
f64,str,str,i64,f64,i64,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,i64
0.0,"""TBL""","""NSH""",2023,0.0,20001,1.0,10.0,61.0,9.0,3.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,59.0,-26.0,59.0,-26.0,-40.914383,40.914383,0.0,0.0,39.698866,"""WRIST""",0.0,0.0,0.0,0.0,…,30.0,28.666667,32.0,33.0,31.0,33.0,26.0,26.0,31.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,0.0,39.698866,59.0,-26.0,26.0,-6.0,-3.4,0.035813,0.249174,0.035816,0.37931,0.277143,0.0227439,0.75132,1.0,1.0,"""TBL""",59.0,2023120001
1.0,"""TBL""","""NSH""",2023,0.0,20001,1.0,11.0,70.0,26.0,9.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,8.0,81.0,8.0,45.0,45.0,85.914383,1.0,11.313708,"""TIP""",0.0,0.0,9.546043,0.0,…,39.0,37.666667,41.0,42.0,40.0,42.0,35.0,35.0,40.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,1.0,11.313708,81.0,8.0,8.0,-6.0,-3.4,0.057705,0.107945,0.051285,0.464358,0.298119,0.020588,0.589712,1.0,1.0,"""TBL""",81.0,2023120001
2.0,"""TBL""","""NSH""",2023,0.0,20001,1.0,13.0,107.0,2.0,11.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,55.0,30.0,55.0,30.0,41.423666,41.423666,0.0,0.0,45.343136,"""SNAP""",0.0,0.0,0.0,0.0,…,23.8,22.333333,26.0,26.0,25.0,26.0,19.0,19.0,26.0,23.8,22.333333,26.0,26.0,25.0,26.0,19.0,19.0,26.0,0.0,45.343136,55.0,30.0,30.0,0.0,-3.8,0.016126,0.236193,0.030269,0.390417,0.301375,0.02562,0.785765,1.0,1.0,"""TBL""",55.0,2023120001
3.0,"""TBL""","""NSH""",2023,0.0,20001,1.0,19.0,177.0,35.0,32.0,1.0,"""HOME""","""AWAYZONE""","""SHOT""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0,-30.0,58.0,-30.0,-44.060809,44.060809,0.0,0.0,43.139309,"""WRIST""",0.0,0.0,0.0,0.0,…,40.0,36.666667,45.0,47.0,47.0,46.0,26.0,26.0,44.0,40.0,36.666667,45.0,47.0,47.0,46.0,26.0,26.0,44.0,1.0,43.139309,58.0,-30.0,30.0,2.0,-3.4,0.0167967,0.198635,0.0336855,0.438052,0.291724,0.021107,0.724471,1.0,1.0,"""TBL""",58.0,2023120001
4.0,"""TBL""","""NSH""",2023,0.0,20001,1.0,21.0,231.0,29.0,19.0,1.0,"""AWAY""","""HOMEZONE""","""MISS""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.0,34.0,64.0,-34.0,-53.673174,53.673174,0.0,0.0,42.201896,"""WRIST""",0.0,0.0,0.0,0.0,…,44.4,47.0,40.5,48.0,48.0,44.0,37.0,46.0,37.0,44.4,47.0,40.5,48.0,48.0,44.0,37.0,46.0,37.0,0.0,42.201896,-64.0,34.0

### 2.a. Adding Dates to Shots

To add dates to shots, we need to query the NHL API to find the game's date. This date needs to be added to a column in the shot, and then the entire dataframe needs to be returned. This will be done in batches so we don't overload the API or have the API stop the program.

In [18]:
def add_dates_to_shots(shots:pl.DataFrame)->pl.DataFrame:
    # Get upper and lower bounds of our seasons (2007-2024 for this instance)
    start = min(shots['season'])
    end = max(shots['season'])

    api_endpt = ''

    # We only need the date for a game a shot takes place in, not the shot itself...
    #  - create groups of unique games, then send those up to the API
    return

In [19]:
shots_with_dates = add_dates_to_shots(shots)

We'll skip this for now and come back to it if we need to. Now we have a completed shots dataset, and go into the dataset for the model.

## 3. Creating the Dataset for the Model

The model will look at each shot and classify it as a clutch shot if it wins the game or sends it to overtime in the last 5 minutes. Then each game will be grouped and a win probability for each game will be computed, and shots will be classified as Clutch<sup>2</sup> when the change in win probability is higher than average (or higher than a certain quantile). To do this we will need to combine the team and shot data

In [20]:
shot_cols_to_use = [
    'shotID',
    'homeTeamCode',
    'awayTeamCode',
    'season',
    'isPlayoffGame',
    'game_id',
    'shooterPlayerId',
    'shooterName',
    'homeTeamWon',
    'composite_game_id',
    'time',
    'period',
    'goal',
    'xGoal',
    'shotGeneratedRebound',
    'xRebound',
    'shotGoalieFroze',
    'xFroze',
    'shotPlayContinuedInZone',
    'xPlayContinuedInZone',
    'shotPlayContinuedOutsideZone',
    'xPlayContinuedOutsideZone',
    'shotPlayStopped',
    'xPlayStopped',
    'shotWasOnGoal',
    'xShotWasOnGoal',
    'isHomeTeam',
    'homeTeamGoals',
    'awayTeamGoals',
    'homeSkatersOnIce',
    'awaySkatersOnIce',
    'awayPenalty1TimeLeft',
    'awayPenalty1Length',
    'homePenalty1TimeLeft',
    'homePenalty1Length'
]

shots_filtered = shots.select(shot_cols_to_use)

In [27]:
team_cols_to_use = ['team', 
               'season', 
               'position', 
               'situation', 
               'games_played', 
               'xGoalsPercentage', 
               'corsiPercentage', 
               'fenwickPercentage', 
               'shotsOnGoalFor', 
               'missedShotsFor',
               'blockedShotAttemptsFor',
               'shotAttemptsFor',
               'goalsFor',
               'goalsAgainst',
               'reboundsFor',
               'reboundGoalsFor'
               ]

teams_filtered = teams.select(team_cols_to_use)

In [22]:
def add_GFg_GAg_scoring_rates_to_teams(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        (pl.col("goalsFor") / pl.col("shotAttemptsFor")).alias("scoringRate"),
        (pl.col("goalsFor") / pl.col("games_played")).alias("GFg"),
        (pl.col("goalsAgainst") / pl.col("games_played")).alias("GAg")
    )

In [23]:
def merge_shots_and_teams(shots_df:pl.DataFrame, teams_df:pl.DataFrame) -> pl.DataFrame:
    # Pivot the DataFrame
    df_pivoted = teams_df.pivot(
        index=["team", "season"],  # Group by team and season
        columns="situation",      # Situations become new columns
        values=["scoringRate"]      # Values in the new columns
    )
    print(df_pivoted)

    # Rename Columns for Clarity
    df_pivoted = df_pivoted.rename(
    {col: f"scoringRate_{col}" for col in df_pivoted.columns if col not in ["team", "season"]}
    )
    
    # Now pivot with GFg and GAg here
    teams_pivoted_2 = teams_df.pivot(
        index=["team", "season"],  # Group by team and season
        columns="situation",      # Situations become new columns
        values=["GFg","GAg"]              # Values in the new columns
    )

    # Join shots with the Team data (just for home team scoring rates!)
    shots_with_home = shots_df.join(
        df_pivoted,
        left_on=["homeTeamCode", "season"],
        right_on=["team", "season"],
        how="left"#,
        #suffix="_home"
    )

    # Join away team goal efficiency with shots (adds away team scoring rates!)
    shots_with_home_and_away = shots_with_home.join(
        df_pivoted,
        left_on=["awayTeamCode", "season"],
        right_on=["team", "season"],
        how="left",
        suffix="_away"
    )

    shots_with_home_2 = shots_with_home_and_away.join(
    teams_pivoted_2,
    left_on=["homeTeamCode", "season"],
    right_on=["team", "season"],
    how="left"
    )
    shots_final_2 = shots_with_home_2.join(
        teams_pivoted_2,
        left_on=["awayTeamCode", "season"],
        right_on=["team", "season"],
        how="left",
        suffix="_away"
    )

    return shots_final_2

In [24]:
def get_combined_df(shots_df:pl.DataFrame, teams_df:pl.DataFrame) -> pl.DataFrame:
    nu_teams_df = add_GFg_GAg_scoring_rates_to_teams(teams_df)
    df = merge_shots_and_teams(shots_df, nu_teams_df)
    return df

In [28]:
df = get_combined_df(shots_filtered, teams_filtered)

/var/folders/b6/zrz7f_vs0k79qphqm5sprhzr0000gn/T/ipykernel_80819/2658159492.py:3: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  df_pivoted = teams_df.pivot(
/var/folders/b6/zrz7f_vs0k79qphqm5sprhzr0000gn/T/ipykernel_80819/2658159492.py:16: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  teams_pivoted_2 = teams_df.pivot(


shape: (522, 7)
┌──────┬────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ team ┆ season ┆ all      ┆ 5on5     ┆ 4on5     ┆ 5on4     ┆ other    │
│ ---  ┆ ---    ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str  ┆ i32    ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞══════╪════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ ANA  ┆ 2008   ┆ 0.054889 ┆ 0.047356 ┆ 0.054545 ┆ 0.078979 ┆ 0.065625 │
│ ARI  ┆ 2008   ┆ 0.048112 ┆ 0.042609 ┆ 0.036585 ┆ 0.056962 ┆ 0.094017 │
│ ATL  ┆ 2008   ┆ 0.060269 ┆ 0.050067 ┆ 0.086957 ┆ 0.07878  ┆ 0.1      │
│ BOS  ┆ 2008   ┆ 0.060558 ┆ 0.049579 ┆ 0.081395 ┆ 0.086957 ┆ 0.122881 │
│ BUF  ┆ 2008   ┆ 0.054702 ┆ 0.047872 ┆ 0.055556 ┆ 0.073082 ┆ 0.07571  │
│ …    ┆ …      ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        │
│ UTA  ┆ 2024   ┆ 0.050445 ┆ 0.040915 ┆ 0.060606 ┆ 0.084071 ┆ 0.128713 │
│ VAN  ┆ 2024   ┆ 0.057807 ┆ 0.049349 ┆ 0.025641 ┆ 0.081301 ┆ 0.130841 │
│ VGK  ┆ 2024   ┆ 0.0595   ┆ 0.05  